# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
geoapify_key = "689fc56ecb0145f39c306bb35442060d"

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = (city_data_df["Max Temp"] >=25) & (city_data_df["Max Temp"]<=30) & (city_data_df["Humidity"]<=50) & (city_data_df["Wind Speed"] <=5)
ideal_weather_cities = city_data_df.loc[ideal_weather, :]

# Drop any rows with null values
ideal_weather_cities=ideal_weather_cities.dropna(how='any')

# Display sample data
ideal_weather_cities


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
170,170,sur,22.5667,59.5289,27.99,40,100,0.49,OM,1666108339
185,185,shakawe,-18.3667,21.8500,29.22,27,86,3.76,BW,1666108350
187,187,gwanda,-20.9333,29.0000,27.66,25,96,3.43,ZW,1666108352
196,196,cabra,37.4725,-4.4421,29.33,30,100,4.53,ES,1666108356
256,256,prieska,-29.6641,22.7474,27.52,31,83,3.22,ZA,1666108388
436,436,lakki marwat,32.6079,70.9114,27.89,20,0,1.53,PK,1666108492
463,463,barroso,-21.1869,-43.9758,25.10,34,83,4.33,BR,1666108504
496,496,tikrit,34.6071,43.6782,29.02,14,0,1.41,IQ,1666108524
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552
561,561,choma,-16.8065,26.9531,26.75,22,94,3.63,ZM,1666108557


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities.copy()
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
170,sur,OM,22.5667,59.5289,40,
185,shakawe,BW,-18.3667,21.8500,27,
187,gwanda,ZW,-20.9333,29.0000,25,
196,cabra,ES,37.4725,-4.4421,30,
256,prieska,ZA,-29.6641,22.7474,31,
436,lakki marwat,PK,32.6079,70.9114,20,
463,barroso,BR,-21.1869,-43.9758,34,
496,tikrit,IQ,34.6071,43.6782,14,
550,pali,IN,25.7667,73.3333,26,
561,choma,ZM,-16.8065,26.9531,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
categories = "hotel"
limit = 20
params = {
    "categories": categories,
    "limit" : limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =     name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sur - nearest hotel: No hotel found
shakawe - nearest hotel: No hotel found
gwanda - nearest hotel: No hotel found
cabra - nearest hotel: No hotel found
prieska - nearest hotel: No hotel found
lakki marwat - nearest hotel: No hotel found
barroso - nearest hotel: No hotel found
tikrit - nearest hotel: No hotel found
pali - nearest hotel: No hotel found
choma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
170,sur,OM,22.5667,59.5289,40,No hotel found
185,shakawe,BW,-18.3667,21.8500,27,No hotel found
187,gwanda,ZW,-20.9333,29.0000,25,No hotel found
196,cabra,ES,37.4725,-4.4421,30,No hotel found
256,prieska,ZA,-29.6641,22.7474,31,No hotel found
436,lakki marwat,PK,32.6079,70.9114,20,No hotel found
463,barroso,BR,-21.1869,-43.9758,34,No hotel found
496,tikrit,IQ,34.6071,43.6782,14,No hotel found
550,pali,IN,25.7667,73.3333,26,No hotel found
561,choma,ZM,-16.8065,26.9531,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha=0.8,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)